In [72]:
import numpy as np

In [73]:
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B_T = np.array([[3, 2, 3],
                [3, 6, 1]])

In [74]:
def approx_equal(x, y, epsilon):
    if y-epsilon < x < y+epsilon:
        return True
    return False

In [75]:
def min_set(v):
    
    idx = []
    small = np.inf
    epsilon = 10e-15
    
    for i in range(len(v)):
        if v[i] < 0:
            continue
        if approx_equal(v[i], small, epsilon):
            idx.append(i)
        elif v[i] < small:
            small = v[i]
            idx = [i]
    
    return idx

In [130]:
def lexico_ratio(tableau, pivot, Crange):
    
    ind_nonpositive = tableau[:, pivot] <= 0
    C = tableau[:, Crange]
    with np.errstate(divide='ignore'):
        C0 = tableau[:, -1] / tableau[:, pivot]
        C0[ind_nonpositive] = np.inf
        row_min = min_set(C0)
    if len(row_min) == 1:
        return row_min[0] 
    
    for i in range(C.shape[1]):
        Ci = (C[:, i] / tableau[:, pivot])[row_min]
        Ci[ind_nonpositive] = np.inf
        row_min = min_set(Ci)
        if len(row_min) == 1:
            return row_min[0]
    
    print("lexico minimum ratio is not found!")

In [108]:
def pivoting(tableau, basic_vars, pivot, Crange):
    
    row_min = lexico_ratio(tableau, pivot, Crange)
    tableau[row_min, :] /= tableau[row_min, pivot]
    for i in range(tableau.shape[0]):
        if i != row_min:
            tableau[i, :] -= tableau[i, pivot] * tableau[row_min, :]
    basic_vars[row_min], pivot = pivot, basic_vars[row_min]
    return pivot

In [109]:
def create_tableau(A, B):
    B_T = B.T
    m, n = A.shape
    tableaus = []
    for i in range(2):
        tableaus.append(np.empty((A.shape[1-i], m+n+1)))
        tableaus[i][:, :m] = [B_T, np.identity(m)][i]
        tableaus[i][:, m:m+n] = [np.identity(n), A][i]
        tableaus[i][:, -1] = 1
    basic_vars_list = [np.arange(m, m+n), np.arange(m)]
    return tableaus, basic_vars_list

In [126]:
def Lemke_Howson(tableaus, basic_vars_list, Crange, init_pivot, return_tableau = False, show_results = False):
    
    m, n = tableaus[1].shape[0], tableaus[0].shape[0]
    pivot = init_pivot
    init_player = int((basic_vars_list[0]==init_pivot).any())
    players = [init_player, 1 - init_player]
    
    while True:
        for i in players:
            pivot = pivoting(tableaus[i], basic_vars_list[i], pivot, Crange[i])
            if show_results:
                print(tableaus[i], basic_vars_list[i])
            if pivot == init_pivot:
                break
        else:
            continue
        break
        
    #summarize the found NE
    normalized = np.empty(m+n)
    out = np.zeros(m+n)
    for i, (start, num) in enumerate(zip([0, m], [m, n])):
        ind = basic_vars_list[i] < start + num if i == 0 else start <= basic_vars_list[i]
        out[basic_vars_list[i][ind]] = tableaus[i][ind, -1]
        s = out[start:start+num].sum()
        if s != 0:
            for j in range(start,start+num):
                normalized[j] = out[j] / s
        else:
            normalized[start:start+num] = np.zeros(num)
    actions = normalized[:m], normalized[m:]
    
    if show_results:
        print(out[:m], out[m:])
        
    if return_tableau:
        return actions, tableaus, basic_vars_list
    
    return actions

In [127]:
tableaus, basic_vars = create_tableau(A, B_T.T)
Crange = basic_vars
Lemke_Howson(tableaus, basic_vars, Crange, 1, show_results = True)

[[ 2.          0.          2.66666667  1.         -0.33333333  0.66666667]
 [ 0.5         1.          0.16666667  0.          0.16666667  0.16666667]] [3 1]
[[ 1.          0.         -0.5         3.          0.          0.5       ]
 [ 0.          1.         -0.83333333  2.          0.          0.16666667]
 [ 0.          0.          0.16666667  0.          1.          0.16666667]] [0 1 4]
[[ 0.75    0.      1.      0.375  -0.125   0.25  ]
 [ 0.375   1.      0.     -0.0625  0.1875  0.125 ]] [2 1]
[[ 1.         -1.5         0.75        0.          0.          0.25      ]
 [ 0.          0.5        -0.41666667  1.          0.          0.08333333]
 [ 0.          0.          0.16666667  0.          1.          0.16666667]] [0 3 4]
[ 0.     0.125  0.25 ] [ 0.08333333  0.16666667]


(array([ 0.        ,  0.33333333,  0.66666667]),
 array([ 0.33333333,  0.66666667]))

In [128]:
#Enumerate all NEs that can be found by Lemke Howson algorithm
def Lemke_Howson_all(A, B):
    
    #define the k of V_k
    k = 0
    
    tableaus, basic_vars = create_tableau(A, B)
    Crange = basic_vars
    m, n = A.shape
    NEs = []
    basic_vars_found = []
    player = (m <= n)
    init_pivot = k
    
    action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, Crange, init_pivot, return_tableau = True)
    NEs.append(action)
    basic_vars_found.append(np.sort(basic_vars[player]))
    
    for a in range(m+n):
        if a == k:
            continue
        tableaus, basic_vars = create_tableau(A, B)
        init_pivot = a
        action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, Crange, init_pivot, return_tableau = True)
        for arr in basic_vars_found:
            if np.array_equal(np.sort(basic_vars[player]), arr):
                break
        else:
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
            
            #start from the found NE by dropping k
            action, tableau, basic_vars = Lemke_Howson(tableaus, basic_vars, Crange, k, return_tableau = True)
            print(action)
            NEs.append(action)
            basic_vars_found.append(np.sort(basic_vars[player]))
    return NEs

In [131]:
Lemke_Howson_all(A, B_T.T)

[(array([ 0.        ,  0.33333333,  0.66666667]),
  array([ 0.33333333,  0.66666667]))]